# **Trích xuất đặc trưng phát hiện Email lừa đảo**


# **1. Mục tiêu**

Với một thư điện tử bất kì, nhóm mong muốn có thể phân loại được thư điện tử đấy là an toàn (legimate)/ không an toàn (phishing)


# **2. Dữ liệu**

Trong project này, nhóm thu thập các tập dữ liệu các loại thư điện tử gồm nội dung và đã được gán nhãn (SpamAssassin, Kaggle, ...). Sau đó nhóm sẽ trích xuất các đặc trưng của các thư điện tử để tạo thành các vector đặc trưng để từ đó làm cơ sở phân loại.


Dataset: Gồm tiêu đề và nội dung, tiêu đề không được phân biệt rõ với nội dung. Chủ yếu xét các đặc trưng theo hình thức email, từ ngữ, và URL


In [1]:
import pandas as pd

In [2]:
#loading the phishing URLs data to dataframe
data0 = pd.read_csv('/Users/HP/projectML/Data/emails.csv') 
data0.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [3]:
data0.shape

(5728, 2)

# **3. Khai thác các đặc trưng**

Trong bước này, các đặc trưng được trích xuất từ bộ dữ liệu Email.

Các đặc trưng trích xuất được phân loại thành

1. Hình thức email
2. Từ ngữ
3. URL


In [4]:
import re
import numpy as np
import matplotlib.pyplot as plt
from spellchecker import SpellChecker

### **3.1 Hình thức email**

Nhiều đặc trưng có thể được trích xuất có thể được coi là đặc trưng cơ sở của hình thức email. Trong số đó, đề cập dưới đây đã được xem xét cho project này.

- Phần mở: Số các từ dear, hi
- Phần kết: Các đặc trung về chữ ký tên, về các từ kết thúc, Kèm thông tin về SDT
- Bố cục : các đặc trưng về Chuyển tiếp, reply

Mỗi đặc trưng này được trình bày và code ở dưới:


In [5]:
def haveFormalWord(email):
    formal_words = ['hi', 'dear', 'best wish', 'regard','dict','reply']
    for word in formal_words:
        if word in email.lower():
            return 0
    return 1 

### **3.2 Từ ngữ**

Nhiều đặc trưng có thể được trích xuất thuộc danh mục này. Trong số đó, đề cập dưới đây đã được xem xét cho project này.

- Chính tả trong từ điển tiếng Anh
- Blacklist:{verify, account, password,access, limitted, risk, security, service, suspend, allow, click, information, risk, credit, minutes, only, last,review,cost, fail, immediate, now, winner, best,any, passion, interest, late, soon, help, update, confirm }
- Kí tự đặc biệt: {%,$,#,?,!}
- Con số: số lượng các số

Mỗi đặc trưng này được trình bày và code ở dưới:


In [6]:
# Check number of misspelled words
def misspelledWord(email):
    spell = SpellChecker()
    words = re.findall(r'\w+', email)
    misspelled = spell.unknown(words)
    return len(misspelled) 

In [7]:
 # Check if the email has typos or bad grammar
def badGrammar(email):
    if len(re.findall(r'[\w]+', email)) > 0:
        typo_ratio = len(re.findall(r'[\w]+', email))/len(email)
        if typo_ratio < 0.3:
            return 1
        else:
            return 0
    else:
        return 0

In [8]:
# Extract common phishing keywords
def keyword(email):
    keywords = [ "login",'account', 'alert', 'verify', 'bank', 'security', "suspend",'fraud', 'urgent', 'warning', 'limited', 'update', 'confirm',  "limited time offer", "urgent",'information']
    for keyword in keywords:
        if keyword in email.lower():
            return 1
        else:
            return 0

In [9]:
# Check if the email asks for personal or financial information
def requestInfo(email):
    # Check if the email asks for personal or financial information
    personal_info = ['password', 'pin', 'ssn', 'social security', 'credit', 'debit', 'card number', 'card expiration', 'cvv']
    for info in personal_info:
        if info in email.lower():
            return 1
        else:
            return 0

In [10]:
# Check if the email includes a sense of Urgency
def senseUrgency(email):
    urgency_words = ['Hurry', 'Urgent', 'Limited time', 'Act now', 'Time running out', 'Last chance', "Don't miss out", 
                     'Limited supply', 'One time offer', 'Only a few left', 'Final sale', 'Today only', 'click']
    for word in urgency_words:
        if word in email.lower():
            return 1
        else:
            return 0

In [11]:
 # Check if the email includes a sense of 
def senseThreat(email):
    threat_words = ['sue', 'arrest', 'lawsuit', 'overdue', 'expired', 'dispute', 'emergency']
    for word in threat_words:
        if word in email.lower():
            return 1
        else:
            return 0

In [12]:
# Check the length of the email
def longEmail(email):
    if len(email) > 500:
        return 1
    else:
        return 0

In [13]:
# Check number of special characters
def numSpecialChar(email):
    num_special_chars = len(re.findall(r'[!@#$%^&*()\-_=+{};:,<.>/?[\]\\|~`]', email))
    return num_special_chars

In [14]:
# Check number of digits
def numDigit(email):
    num_digits = len(re.findall(r'\d', email))
    return num_digits

### **3.3 URL**

Nhiều đặc trưng có thể được trích xuất thuộc danh mục này. Trong số đó, đề cập dưới đây đã được xem xét cho project này.

- Tên miền: Số hay là chữ
- Độ dài miền: Ngắn/dài
- Chuẩn HTTP: có/ không
- Số dot trong domain
- Redirection "//" trong URL
- Ký tự '-' trong miền

Mỗi đặc trưng này được trình bày và code ở dưới:


In [15]:
def numLink(email):
    # Check if number of links
    num_of_links = len(re.findall(r"http", email))
    return num_of_links

In [16]:
def suspiciousLink(email):
    # Check if the email has suspicious link
    # links = re.find(r"(http?://\S+)", email)
    # if links:
    #     for link in links:
    #         if  '-' in link or 'paypal' in link or 'bank' in link or 'irs' in link or 'amazon' in link or 'ebay' in link:
    #             return 1
    #         else:
    #             return 0
    # else:
    #     return 0
    num_suspicious_link = 0
    for content in email:
        urls = re.findall(r'\b(http? : / / (?:[-\w. ]|(?:%[\da-fA-F]{2}))+\. (?:[a-z]{2,5})|(?: / [\w\.-]*)*/)\b', content)
        num_suspicious_link +=1
    return num_suspicious_link

In [17]:
def numDot(email):
    num_dot = 0
    # Check if the link in email has dot
    links = re.findall(r"http", email)
    for link in links:
        num_dot += len(re.findall(r".", link))
    return num_dot

# **4. Danh sách các đặc trưng**

Tạo một danh sách và một hàm gọi các hàm khác và lưu trữ tất cả các đặc trưng của email trong danh sách. Chúng tôi sẽ trích xuất các đặc trưng của từng email và thêm vào danh sách này.


In [18]:
def extract_features(email):
    features = []
    
    features.append(haveFormalWord(email))
    features.append(misspelledWord(email))
    features.append(badGrammar(email))
    features.append(keyword(email))
    features.append(requestInfo(email))
    features.append(senseUrgency(email))
    features.append(senseThreat(email))
    features.append(longEmail(email))
    features.append(numSpecialChar(email))
    features.append(numDigit(email))
    features.append(numLink(email))
    features.append(suspiciousLink(email))
    features.append(numDot(email))
    
    return features

In [ ]:
#Extracting the feautres & storing them in a list
features_list = []
# for email in data0:
for i in range(2000):
    email = data0['text'][i]
    # Add the email features to the list of all features
    features_list.append(extract_features(email))

# **5. Tệp dữ liệu cuối cùng**

Bây giờ, chúng ta có một khung dữ liệu và xuất dữ liệu sang tệp csv để đào tạo Machine Learning được thực hiện trong phần sau.


In [ ]:
# Convert the list of features into a pandas DataFrame
feature_names = ['have_formal_word', 'num_misspelled_words','bad_grammar', 'keyword', 'requests_personal_info', 'sense_of_urgency', 'sense_of_threat', 'long_email', 'number_special_chars', 'number_digits', 'number_links','suspicious_link', 'number_dot_in_link']
df = pd.DataFrame(features_list, columns= feature_names)
df.head()

In [28]:
#Plotting the data distribution
df.hist(bins = 50,figsize = (15,15))
plt.show()

ValueError: hist method requires numerical or datetime columns, nothing to plot.

In [ ]:
# Storing the extracted email fatures to csv file
df.to_csv('email_processed.csv', index= False)